# Dados UFRN - Análise


## Descrição do projeto

## Descrição dos dados


## Autores


## Vídeo explicativo



Importando bibliotecas a serem usadas

In [42]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#conda install -c conda-forge seaborn

Abertura de dados

In [108]:
#Carregando arquivo referente as unidades
data_unidades = pd.read_csv("Dados/unidades/unidades.csv", sep=';',
                            usecols=["id_unidade", "nome_unidade", "sigla", "municipio","id_unidade_gestora",
                                     "id_unidade_responsavel", "unidade_responsavel", "classificacao_unidade"])


#Selecionando somente os institutos e os centros
centro_ou_inst = [nome.startswith("CENTRO") or nome.startswith("INSTITUTO") for nome in data_unidades.nome_unidade]
data_unidades = data_unidades[centro_ou_inst]

#Carregando arquivo referente aos gastos
data_gastos = pd.read_csv("Dados/unidades/gastos-por-unidade.csv", 
                          sep=';',
                          usecols=["id_unidade", "natureza_despesa", "valor"])

#Convertendo coluna de String para float
data_gastos.valor = data_gastos.valor.replace('[R | ' ' | \$,.]', '', regex=True).astype(float) / 100

#Unindo as tabelas
dataM_unit_gas = pd.merge(data_unidades, data_gastos, on="id_unidade")

#Criando pivot table para somar gasto por unidade
ptable = pd.pivot_table(dataM_unit_gas, index=["id_unidade", "nome_unidade"], values=["valor"], aggfunc=np.sum)

#Salvando essa pivot table num DataFrame
data_gastos_ok = pd.DataFrame(ptable.to_records())

,id_unidade,nome_unidade,valor
0,439,CENTRO DE CIENCIAS EXATAS E DA TERRA,1.043835e+08
1,440,CENTRO DE BIOCIENCIAS,6.436635e+07
2,441,CENTRO DE CIENCIAS DA SAUDE,1.184980e+08
3,442,"CENTRO DE CIENCIAS HUMANAS, LETRAS E ARTES",6.963006e+07
4,443,CENTRO DE CIENCIAS SOCIAIS APLICADAS,8.373748e+07
5,445,CENTRO DE TECNOLOGIA,7.818721e+07
6,1482,CENTRO DE ENSINO SUPERIOR DO SERIDO,2.611328e+07
7,5965,CENTRO DE EDUCACAO,1.867217e+07
8,6026,INSTITUTO DO CEREBRO,3.303266e+07
9,6069,INSTITUTO METROPOLE DIGITAL,4.944037e+07


In [112]:
#Carregando arquivo dos docentes (link entre turmas e unidades gestoras)
link_docentes = pd.read_csv("Dados/docentes/docentes.csv",
                           sep=';',
                           usecols=["id_servidor", "id_unidade_lotacao"])

,id_servidor,id_unidade_lotacao
0,5753261,1452
1,5752617,1452
2,22092,351
3,2263697,1452
4,5751986,351
5,23477,284
6,5758792,351
7,5752871,351
8,22880,284
9,5752921,284


In [140]:
#Carregando arquivos de semestre e linkando

for i in range(2017,2018):
    for j in range(1,3):
        #Carregando strings com o caminho dos dados a serem carregados
        file_matricula = "Dados/matriculas/" + (str(i) + str(j)) + ".csv"
        file_turmas = "Dados/turmas/" + (str(i) + str(j)) + ".csv"
        
        #Carregando arquivo de matriculas de certo semestre para DataFrame
        data_matriculas = pd.read_csv(file_matricula, sep=';',
                                      usecols=["id_turma", "discente", "descricao"])
        
        
        
        
        #Carregando arquivo de turmas de certo semestre para DataFrame
        data_turmas = pd.read_csv(file_turmas, sep=';',
                                  usecols=["id_turma", "id_docente_interno", "nivel_ensino",
                                           "ano", "periodo","situacao_turma"])
        
        #Restringindo a busca para o ano procurado
        data_turmas = data_turmas[data_turmas.ano == i]
        
        #Restringindo a busca para o periodo procurado
        data_turmas = data_turmas[data_turmas.periodo == j]
        
        #Restringindo a busca para o curso superior
        data_turmas = data_turmas[data_turmas.nivel_ensino == "GRADUAÇÃO"]
        
        #Restringindo a busca para turmas consolidadas
        data_turmas = data_turmas[data_turmas.situacao_turma == "CONSOLIDADA"]
        
        #Dando merge nas tabelas turmas e matriculas
        dataM_mat_tur = pd.merge(data_turmas, data_matriculas, on="id_turma")
        
        ptable = pd.pivot_table(dataM_mat_tur, index=["id_turma","descricao"],
                                values="ano", aggfunc="count").rename(columns={'ano': 'Contagem'})
        
        #Salvando essa pivot table num DataFrame
        data_cont_situ = pd.DataFrame(ptable.to_records())
        
        print(data_cont_situ)
        
        #Corrigindo nome de coluna, para possibilitar merge
        dataM_mat_tur.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)
        dataM_mt_doc = pd.merge(dataM_mat_tur, link_docentes, on="id_servidor")
        

       id_turma                         descricao  Contagem
0      57593679                          APROVADO        90
1      57593679                         CANCELADO         4
2      57593679                       DESISTENCIA         4
3      57593679                          EXCLUIDA         5
4      57593679                        INDEFERIDO         8
5      57593679  REPROVADO POR MÉDIA E POR FALTAS         9
6      57593679                          TRANCADO         9
7      57593680                          APROVADO        42
8      57593680                 APROVADO POR NOTA        21
9      57593680                         CANCELADO        11
10     57593680                       DESISTENCIA         1
11     57593680                          EXCLUIDA         2
12     57593680                        INDEFERIDO         1
13     57593680                         REPROVADO        24
14     57593680                REPROVADO POR NOTA         3
15     57593680                         

In [98]:
sns.set()
data_gastos
# The four preset contexts, in order of relative size, are paper, notebook, talk, and poster
sns.set_context("notebook")
data_gastos.valor = data_gastos.valor.astype(float)
# plot a univariate distribution of observations.
#sns.kdeplot(int(data_gastos["valor"]))

ValueError: could not convert string to float: 'R$           389,20'